# **🌸 Mental Health Assistant Chatbot Project 🌸**

This project aims to develop a **Mental Health Assistant Chatbot** designed to offer emotional support and companionship. Utilizing **sentiment analysis** and **natural language generation**, the chatbot listens to user input, detects emotional tone, and provides empathetic, helpful responses. It uses the **Llama-2** language model for generating text and **DistilBERT** for sentiment analysis, ensuring that responses are tailored to the user's mood. The chatbot is implemented with **Gradio**, offering a user-friendly interface that allows for interactive conversations. The focus is on providing a cozy, supportive environment where users can share their feelings and receive positive reinforcement.






# **Step 1: Install the Required Libraries**

We start by installing the necessary Python packages like torch, transformers, and gradio. These will be used for building the sentiment analysis and text generation system.

In [ ]:
# Step 1: Install the necessary libraries
!pip install torch torchvision torchaudio transformers sentencepiece gradio sentence-transformers accelerate

# **Step 2: Import the Libraries**
After installing, we import the required modules. These include torch for handling tensor operations, transformers for the language models, and gradio for building the interactive user interface.

In [ ]:
# Step 2: Import required libraries
import torch
from transformers import pipeline, AutoModelForCausalLM, AutoTokenizer
import gradio as gr


# **Step 3: Initialize Sentiment Analysis and Language Model**
Here, we initialize two components:



*   Sentiment Analysis: Using a pre-trained model (distilbert-base-uncased-finetuned-sst-2-english) to detect user emotions.
*  Llama-2 Chat Model: A large language model for generating responses to user input.





In [ ]:
# Step 3: Initialize the sentiment analyzer and language model
emotion_detector = pipeline("sentiment-analysis", model="distilbert-base-uncased-finetuned-sst-2-english")
model_identifier = "NousResearch/Llama-2-7b-chat-hf"  # The model ID for Llama 2
llama_model = AutoModelForCausalLM.from_pretrained(model_identifier, torch_dtype=torch.float16, device_map="auto")
llama_tokenizer = AutoTokenizer.from_pretrained(model_identifier)


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

# **Step 4: Setup the Text Generation Pipeline**
This pipeline is responsible for generating dynamic responses using the Llama model. We adjust the parameters like max_length, temperature, and top_p for better text generation quality.

In [ ]:
# Step 4: Set up the response generation pipeline
response_producer = pipeline(
    "text-generation", model=llama_model, tokenizer=llama_tokenizer,
    torch_dtype=torch.float16, device_map="auto",
    max_length=4096, do_sample=True, temperature=0.6, top_p=0.9
)

# **Step 5: Create the Response Generation Function**
This function generates a response based on user input. It adjusts the maximum response length based on the input length to create coherent replies.

In [ ]:
# Step 5: Function to generate a response using the Llama model
def create_response(user_input_text):
    input_length = len(user_input_text.split())
    max_response_length = min(4096, input_length * 10 + 200)
    return response_producer(user_input_text, max_length=max_response_length, do_sample=True)[0]['generated_text'].strip()


# **Step 6: Analyze the Sentiment of the User's Input**
This function takes user input and detects whether the sentiment is positive or negative. It provides an appropriate emotional response based on the sentiment score.

In [ ]:
# Step 6: Function to analyze the sentiment of user input
def assess_emotion(user_text):
    sentiment = emotion_detector(user_text)[0]
    category, confidence = sentiment['label'], sentiment['score']

    if category == "NEGATIVE":
        return "I'm sorry you're having a tough time. I'm here for you." if confidence > 0.9 else "Things may seem difficult. I’m listening."
    elif category == "POSITIVE":
        return "It's great to see positivity! Keep it up!" if confidence > 0.9 else "Glad to see you're feeling better."
    else:
        return "I'm here to listen, no matter what you're going through."


# **Step 7: Clean Up Generated Responses**
Sometimes generated responses may contain redundant phrases. This function helps in removing them to make the conversation more natural.

In [ ]:
# Step 7: Function to clean redundant phrases in the generated text
def remove_redundant_phrases(user_text, generated_text):
    unwanted_phrases = ["It seems rough.", "Take a deep breath."]
    for phrase in unwanted_phrases:
        generated_text = generated_text.replace(phrase, "").strip()

    return generated_text


# **Step 8: Combine Sentiment Analysis and Text Generation**
This function combines sentiment analysis with text generation to provide a supportive, dynamic response to the user. It uses both emotion detection and Llama-generated responses.

In [ ]:
# Step 8: Function to combine sentiment analysis and response generation
def reply_to_user(user_text):
    sentiment_message = assess_emotion(user_text)
    generated_response = create_response(user_text)
    final_response = remove_redundant_phrases(user_text, generated_response)
    return f"{sentiment_message} {final_response}".strip()


# **Step 9: Manage Conversation History**
To make the conversation feel more natural, we store the conversation history. This function handles the user's input, generates a response, and maintains a history of the conversation.

In [ ]:
# Step 9: Maintain conversation history
conversation_history = []

# Main function to manage the conversation flow
def manage_conversation(user_text):
    global conversation_history

    if user_text.lower() == "clear":
        conversation_history = []
        return "Conversation history cleared."

    bot_reply = reply_to_user(user_text)
    conversation_history.append(f"USER: {user_text}\nBOT: {bot_reply}")

    return "\n\n".join(conversation_history)


# **Step 10: Build the Gradio Chat Interface**
Finally, we use gradio to create a simple chat interface. The user can interact with the bot, and the chat history will be displayed on the screen.

In [ ]:
# Step 10: Gradio interface setup
chat_interface = gr.Interface(
    fn=manage_conversation,
    inputs="text",
    outputs="text",
    title="Your Well-Wisher",
    description="Talk to me and get supportive responses based on your feelings. Type 'clear' to start a new conversation.",
    allow_flagging="never"
)

# Launch the Gradio interface with debugging enabled
chat_interface.launch(debug=True)


Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://5ee9807853c5756d3e.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7861 <> https://5ee9807853c5756d3e.gradio.live
